In [17]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

In [19]:
files = os.listdir("/content/drive/My Drive/Data")
files

['full_numpy_bitmap_apple.npy',
 'full_numpy_bitmap_bowtie.npy',
 'full_numpy_bitmap_candle.npy',
 'full_numpy_bitmap_door.npy',
 'full_numpy_bitmap_envelope.npy',
 'full_numpy_bitmap_fish.npy',
 'full_numpy_bitmap_guitar.npy',
 'full_numpy_bitmap_ice cream.npy',
 'full_numpy_bitmap_lightning.npy',
 'full_numpy_bitmap_moon.npy',
 'full_numpy_bitmap_mountain.npy',
 'full_numpy_bitmap_star.npy',
 'full_numpy_bitmap_tent.npy',
 'full_numpy_bitmap_toothbrush.npy',
 'full_numpy_bitmap_wristwatch.npy']

In [0]:
x = []
x_load = []
y = []
y_load = []

In [0]:
def load_data():
    count = 0
    for file in files:
        file = "/content/drive/My Drive/Data/" + file
        x = np.load(file)
        x = x.astype('float32') / 255.
        x = x[0:10000, :]
        x_load.append(x)
        y = [count for _ in range(10000)]
        count += 1
        y = np.array(y).astype('float32')
        y = y.reshape(y.shape[0], 1)
        y_load.append(y)
    return x_load, y_load

In [0]:
features, labels = load_data()

In [0]:
features = np.array(features).astype('float32')

In [0]:
labels = np.array(labels).astype('float32')

In [0]:
features=features.reshape(features.shape[0]*features.shape[1],features.shape[2])

In [0]:
labels=labels.reshape(labels.shape[0]*labels.shape[1],labels.shape[2])

In [0]:
with open("features", "wb") as f:
    pickle.dump(features, f, protocol=4)
with open("labels", "wb") as f:
    pickle.dump(labels, f, protocol=4)

In [0]:
def keras_model(image_x, image_y):
    num_of_classes = 15
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list

In [0]:
def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels

In [0]:
def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

In [0]:
def prepress_labels(labels):
    labels = np_utils.to_categorical(labels)
    return labels

In [32]:
def main():
    features, labels = loadFromPickle()
    # features, labels = augmentData(features, labels)
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,
                                                        test_size=0.1)
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    model, callbacks_list = keras_model(28,28)
    print_summary(model)
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64,
              callbacks=[TensorBoard(log_dir="QuickDraw")])
    model.save('QuickDraw.h5')

main()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)             